## Objetivo: Analise de sentimentos com redes neurais (NLP)

### Imports

In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from nltk.tokenize import word_tokenize
import nltk
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding

Using TensorFlow backend.


### Dataset

In [2]:
corpus = [
    # Positive Reviews
    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews
    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

# labels
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

### Pre-processamento

In [3]:
# lista de todas as palavras
all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

unique_words = set(all_words)
vocab_length = 50
print(len(unique_words))

embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences)

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

45
[[22, 7, 5, 41, 47], [40, 40, 20, 36, 18, 40, 49], [46, 11, 47, 49, 7, 14], [33, 19], [23, 34, 29, 45, 18, 40, 49], [30, 15, 36, 48], [11, 8, 35, 15, 18, 47], [49, 7, 15, 25, 47], [4, 10], [14, 15, 29], [31, 1], [49, 20, 30, 9], [18, 34, 16, 40, 40, 47], [40, 47, 20, 4], [18, 43, 16, 35], [40, 10, 7, 31]]
[[22  7  5 41 47  0  0]
 [40 40 20 36 18 40 49]
 [46 11 47 49  7 14  0]
 [33 19  0  0  0  0  0]
 [23 34 29 45 18 40 49]
 [30 15 36 48  0  0  0]
 [11  8 35 15 18 47  0]
 [49  7 15 25 47  0  0]
 [ 4 10  0  0  0  0  0]
 [14 15 29  0  0  0  0]
 [31  1  0  0  0  0  0]
 [49 20 30  9  0  0  0]
 [18 34 16 40 40 47  0]
 [40 47 20  4  0  0  0]
 [18 43 16 35  0  0  0]
 [40 10  7 31  0  0  0]]


### Modelo

In [4]:
model = Sequential()

model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 20)             1000      
_________________________________________________________________
flatten (Flatten)            (None, 140)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6930 - acc: 0.3750
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6897 - acc: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 999us/step - loss: 0.6863 - acc: 0.6875
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6830 - acc: 0.7500
Epoch 5/1

1/1 [==============================] - 0s 997us/step - loss: 0.3626 - acc: 1.0000
Epoch 81/100
1/1 [==============================] - 0s 995us/step - loss: 0.3579 - acc: 1.0000
Epoch 82/100
1/1 [==============================] - 0s 998us/step - loss: 0.3532 - acc: 1.0000
Epoch 83/100
1/1 [==============================] - 0s 0s/step - loss: 0.3485 - acc: 1.0000
Epoch 84/100
1/1 [==============================] - 0s 0s/step - loss: 0.3439 - acc: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 0s/step - loss: 0.3393 - acc: 1.0000
Epoch 86/100
1/1 [==============================] - 0s 998us/step - loss: 0.3347 - acc: 1.0000
Epoch 87/100
1/1 [==============================] - 0s 0s/step - loss: 0.3301 - acc: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 0s/step - loss: 0.3256 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 998us/step - loss: 0.3211 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 0s/step - loss: 0.3167 

### Teste com Glove

In [5]:
embeddings_dictionary = dict()
glove_file = open('data/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()


word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

model.fit(padded_sentences, sentiments, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 100)            5000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 701       
Total params: 5,701
Trainable params: 701
Non-trainable params: 5,000
_________________________________________________________________
None
Epoch 1/100
1/1 [==============================] - 0s 998us/step - loss: 0.6701 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 997us/step - loss: 0.6475 - acc: 0.5625
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6256 - acc: 0.5625
Epoch 4/100
1/1 [==============================] - 0s 998us/step - loss: 0.6045 - acc: 0.6875
E

1/1 [==============================] - 0s 997us/step - loss: 0.1060 - acc: 1.0000
Epoch 81/100
1/1 [==============================] - 0s 996us/step - loss: 0.1044 - acc: 1.0000
Epoch 82/100
1/1 [==============================] - 0s 0s/step - loss: 0.1029 - acc: 1.0000
Epoch 83/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1015 - acc: 1.0000
Epoch 84/100
1/1 [==============================] - 0s 0s/step - loss: 0.1000 - acc: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 998us/step - loss: 0.0986 - acc: 1.0000
Epoch 86/100
1/1 [==============================] - 0s 0s/step - loss: 0.0973 - acc: 1.0000
Epoch 87/100
1/1 [==============================] - 0s 993us/step - loss: 0.0959 - acc: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 0s/step - loss: 0.0946 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0933 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 984us/step - loss: 0.092